In [1]:
! pip install pyodbc


You should consider upgrading via the 'c:\users\fpa\proyectos\asm-web\myenv\scripts\python.exe -m pip install --upgrade pip' command.


In [2]:
import pyodbc
import base64

In [3]:
# Definir parámetros de conexión
server = "192.168.7.128"  # e.g., "localhost" or "your_server_name\instance_name"
database = "UserManagement"
username = "sa"
password = "P@ssw0rd"

# Definición de string de conexión
connection_string = f"DRIVER={{SQL Server}};SERVER={server};DATABASE={database};UID={username};PWD={password}"

# Conexión a la base de datos
connection = pyodbc.connect(connection_string)

# Apertura de conexión con SQL
cursor = connection.cursor()

# Si hemos guardado la contraseña codificada
#pwd_cod = base64.b64encode(request.password.encode("utf-8")).decode("utf-8")

#
request_user = "Fio"
query = f"SELECT * FROM Usuarios"
print(query)
cursor.execute(query)

# 
result = cursor.fetchall()
result

SELECT * FROM Usuarios


[('AdminNextret', 'f7ff4c6d1d24ffb0712f82f70ef5cc9e8ba2b650cb3f012c072b3726341f3f46', '967f7f0f8b931434')]

In [4]:
import hashlib
import os
import secrets
import pandas as pd


def query_oracle_select(connection, query):
    print("query", query)
    result_df = pd.read_sql(query, connection)

    return result_df


def query_oracle_insert(connection, query):
    cursor = connection.cursor()
    print("query", query)
    
    cursor.execute(query)

    # Commit the transaction
    connection.commit()
    cursor.close()


def create_user(username, password):
    # Crear hex aleatorio
    salt_hex= secrets.token_hex(8)
    salt = bytes.fromhex(salt_hex)
    print("salt", salt)
    # Concatenar salt y contraseña
    password_salt = password.encode() + salt
    
    # Crear hash
    sha256_hash = hashlib.sha256(password_salt).hexdigest()
    print("SHA-256 Hash with Salt:", sha256_hash)
    
    query = f"Insert into Usuarios (Username, PasswordHash, Salt) VALUES ('{username}', '{sha256_hash}' , '{salt_hex}');"
    query_oracle_insert(connection, query)
    print("query_result")

    
def create_roles(connection, username, roles):
    values_query = [f"('{username}', '{rol}')"  for rol in roles]
    values_query = (',').join(values_query)
    query = f"insert into Usuarios_roles (Username, Rol) VALUES {values_query} ;"
    query_oracle_insert(connection, query)
    
def create_tags(connection, username, tags):
    values_query = [f"('{username}', '{tag}')"  for tag in tags]
    values_query = (',').join(values_query)
    query = f"insert into Usuarios_tags (Username, Tag) VALUES {values_query} ;"
    query_oracle_insert(connection, query)
    
def check_user(connection, username, password):
    cursor = connection.cursor()
    
    query = f"Select * from Usuarios  where Username = '{username}'"
    query_result = query_oracle_select(connection, query)
    #print("query_result", query_result)
    if(query_result.empty):
        print("No existe el usuario!!")
    else:
        print("Existe el usuario")
        password_hashed = query_result['PasswordHash'].iloc[0]
        print(password_hashed)
        salt_hex = query_result['Salt'].iloc[0]
        print(salt_hex)
        salt = bytes.fromhex(salt_hex)
        
        given_password_salt = password.encode() + salt
    
        # Crear hash
        given_sha256_hash = hashlib.sha256(given_password_salt).hexdigest()
        print(given_sha256_hash)
        if given_sha256_hash == password_hashed:
            query = f"select * from Usuarios_tags where username = '{username}'"
            tags = query_oracle_select(connection, query)['Tag']
            tags = list(tags)
            
            query = f"select * from Usuarios_roles where username = '{username}'"
            roles = query_oracle_select(connection, query)['Rol']
            roles = list(roles)
            
            user = {"username": username, "roles": roles, "tags": tags}

            print("Contraseña correcta!")
        else:
            print("Contraseña incorrecta!")
        
        
        

In [13]:
check_user(connection, "AdminNextret", "AdminNextret!")

query Select * from Usuarios  where Username = 'AdminNextret'
Existe el usuario
f7ff4c6d1d24ffb0712f82f70ef5cc9e8ba2b650cb3f012c072b3726341f3f46
967f7f0f8b931434
f7ff4c6d1d24ffb0712f82f70ef5cc9e8ba2b650cb3f012c072b3726341f3f46
Contraseña correcta!


In [31]:
query = f"select * from Usuarios_tags where username = '{username}'"
tags = query_oracle_select(connection, query)['Tag']
tags = list(tags)

query select * from Usuarios_tags where username = 'AdminNextret'


In [34]:
query = f"select * from Usuarios_roles where username = '{username}'"
roles = query_oracle_select(connection, query)['Rol']
roles = list(roles)
roles

query select * from Usuarios_roles where username = 'AdminNextret'


['admin']

In [6]:
create_user("AdminNextret", "AdminNextret!")

salt b'\x96\x7f\x7f\x0f\x8b\x93\x144'
SHA-256 Hash with Salt: f7ff4c6d1d24ffb0712f82f70ef5cc9e8ba2b650cb3f012c072b3726341f3f46
query Insert into Usuarios (Username, PasswordHash, Salt) VALUES ('AdminNextret', 'f7ff4c6d1d24ffb0712f82f70ef5cc9e8ba2b650cb3f012c072b3726341f3f46' , '967f7f0f8b931434');
query_result <function query_oracle_insert at 0x000001A85609F9D0>


In [22]:
create_roles(connection, "AdminNextret", ["admin"])

query insert into Usuarios_roles (Username, Rol) VALUES ('AdminNextret', 'admin') ;


In [27]:
create_tags(connection, "AdminNextret", ["web", "asm", "lanprova"])

query insert into Usuarios_tags (Username, Tag) VALUES ('AdminNextret', 'web'),('AdminNextret', 'asm'),('AdminNextret', 'lanprova') ;


In [19]:
roles = ['admin']
username = "f"
roles_query = [f"('{username}', '{rol}')"  for rol in roles]
roles_query = (',').join(roles_query)
query = f"insert into Usuarios_roles (Username, Rol) VALUES {roles_query} ;"
query

"insert into Usuarios_roles (Username, Rol) VALUES ('f', 'admin') ;"

In [19]:
import hashlib
import os

# Simulated stored password hash and salt (you would fetch these from your database)
stored_password_hash = "2f04dc6c9b3d49f42c4c0494b66363c3f3263f74e759dc38b2260951cdf92218"  # Replace with your actual stored hash
salt = b'\xff\xfd\xb7m\xe7]\x10\x05!\x9f\x85#\xea\xed\x81\x17'  # Replace with your actual salt
print(salt)

# User-entered password to verify
entered_password = "AdminNextret!"

# Concatenate the entered password and stored salt
password_salt = entered_password.encode() + salt

# Create a SHA-256 hash of the entered password with the stored salt
entered_password_hash = hashlib.sha256(password_salt).hexdigest()

# Compare the entered password hash with the stored hash
if entered_password_hash == stored_password_hash:
    print("Password is correct!")
else:
    print("Password is incorrect.")

ÿý·mç]!#êí


TypeError: can't concat str to bytes

In [5]:
query = f"Select * from Usuarios"
query_result = query_oracle_select(connection, query)
query_result

query Select * from Usuarios


,Username,PasswordHash,Salt
0,AdminNextret,f7ff4c6d1d24ffb0712f82f70ef5cc9e8ba2b650cb3f01...,967f7f0f8b931434


In [8]:
if(not query_result.empty):
    print("Not empty")
    users_list = []

    for index, usuario in query_result.iterrows():
        username = usuario['Username']
        query = f"select * from Usuarios_tags where username = '{username}'"
        tags = query_oracle_select(connection, query)['Tag']
        tags = list(tags)
        print("tags", tags)
        query = f"select * from Usuarios_roles where username = '{username}'"
        roles = query_oracle_select(connection, query)['Rol']
        roles = list(roles)
        print("roles", roles)

        user = {"username": username, "roles": roles, "tags": tags}
        users_list.append(user)
        print(users_list)
else: 
    user = {"username": "", "roles": [], "tags": []}
    users_list.append(user)

    #return users_list

Not empty
query select * from Usuarios_tags where username = 'AdminNextret'
tags ['web', 'asm', 'lanprova']
query select * from Usuarios_roles where username = 'AdminNextret'
roles ['admin']
[{'username': 'AdminNextret', 'roles': ['admin'], 'tags': ['web', 'asm', 'lanprova']}]
